In [ ]:
import numpy as np
from cloth_tools.annotation.grasp_annotation import grasp_hanging_cloth_pose
from cloth_tools.dataset.format import save_grasp_pose
from cloth_tools.dataset.upload import upload_grasp

grasp_pose_fixed = grasp_hanging_cloth_pose(np.array([0, 0, 0.5]), np.array([1, 0, 0]), 0.0)
sample_id = "2024-04-23_10-20-07-968516"
grasps_dir = f"../data/grasps_{sample_id}"

grasp_pose_file = save_grasp_pose(grasps_dir, grasp_pose_fixed)

team_name = "dev_team"
server_url = "https://robotlab.ugent.be"

upload_grasp(grasp_pose_file, team_name, sample_id, server_url)

In [ ]:
import os
import urllib
import zipfile
from pathlib import Path

import requests
from cloth_tools.dataset.format import COMPETITION_OBSERVATION_FILENAMES
from tqdm import tqdm


def get_dev_latest_observation_start_url(server_url: str) -> str:
    url = server_url + "/dev_latest_observation_start_dir"  # simple API to get the latest observation directory
    response = requests.get(url)
    return server_url + "/" + response.text


def dev_download_latest_observation(dataset_dir: str, server_url: str) -> tuple[str, str]:
    """Download the latest start observation's files from the Cloth Competition server.
    Also returns the sample ID, which must be used when uploading a grasp.json file.

    Args:
        dataset_dir: The directory to save observation files to.
        server_url: URL of the server to download from.

    Returns:
        A tuple containing the path to the observation directory and the sample ID.
    """
    files_to_retrieve = COMPETITION_OBSERVATION_FILENAMES
    observation_dir_url = get_dev_latest_observation_start_url(server_url)

    sample_dirname = observation_dir_url.split("/")[-2]
    sample_datetime = sample_dirname.split("sample_")[-1]

    sample_dir = Path(dataset_dir) / sample_dirname
    observation_start_dir = sample_dir / "observation_start"
    os.makedirs(observation_start_dir, exist_ok=True)

    for _, filename in files_to_retrieve.items():
        url = observation_dir_url + "/" + filename
        response = requests.get(url)
        filepath = os.path.join(observation_start_dir, filename)
        with open(filepath, "wb") as f:
            f.write(response.content)

    return str(observation_start_dir), sample_datetime


observation_start_dir, sample_id_latest = dev_download_latest_observation("../data", server_url)

print(observation_start_dir)
print(sample_id_latest)

In [ ]:
import matplotlib.pyplot as plt
from cloth_tools.dataset.format import load_competition_observation

observation = load_competition_observation(observation_start_dir)

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(observation.image_left)
plt.title("Left image")
plt.subplot(1, 2, 2)
plt.imshow(observation.image_right)
plt.title("Right image")
plt.show()

In [ ]:
upload_grasp(grasp_pose_file, team_name, sample_id_latest, server_url)